# FastKafka

This notebook will demonstrate the capabilities and developed functionalities in FastKafka project


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/airtai/fastkafka/blob/64-colab-based-tutorial/nbs/guides/Guide_00_FastKafka_Demo.ipynb)

## Installing fastkafka library

To install fastkafka, run: `pip install fastkafka` in your terminal

In [ ]:
try:
    import fastkafka
except ImportError:
    #!pip install fastkafka==0.1.0
    !pip install "fastkafka @ git+https://github.com/airtai/fastkafka@05a30a4dbc0f0b47f6f8ad0f0ab114c76cc42780"

## FastKafka demo

Now we will create a fastkafka application containing a Model that will ingest data samples from one Kafka topic (input_data) and produce predictions to another Kafka topic (predictions)

### Preparing the demo model

First we will prepare our model with the Iris dataset so that we can demonstrate the preditions using FastKafka

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X, y = load_iris(return_X_y=True)
model = LogisticRegression(random_state=0, max_iter=500).fit(X, y)
x = X[[0, 55, -1]]
print(x)
print(model.predict(x))

[[5.1 3.5 1.4 0.2]
 [5.7 2.8 4.5 1.3]
 [5.9 3.  5.1 1.8]]
[0 1 2]


Now, we need to model the input and prediction messages that will be sent to the Kafka broker

In [ ]:
from pydantic import BaseModel, NonNegativeFloat, Field

class IrisInputData(BaseModel):
    sepal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal length in cm"
    )
    sepal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal width in cm"
    )
    petal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal length in cm"
    )
    petal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal width in cm"
    )


class IrisPredictionData(BaseModel):
    species: str = Field(..., example="setosa", description="Predicted species")

Now, lets prepare our prediction FastKafka app.

In [ ]:
from fastkafka.application import FastKafka

kafka_app = FastKafka()

iris_species = ["setosa", "versicolor", "virginica"]

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = model.predict([
          [msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]
        ])[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPredictionData:
    prediction = IrisPredictionData(species=iris_species[species_class])
    return prediction

Lets run the test by sending a message to the running app that now encapsulates the Iris classification model:

In [ ]:
from fastkafka.application import Tester

msg = IrisInputData(
    sepal_length=0.1,
    sepal_width=0.2,
    petal_length=0.3,
    petal_width=0.4,
)

# Start Tesr app and create local Kafka broker for testing
async with Tester(kafka_app) as tester:
    # Send IrisInputData message to input_data topic
    await tester.to_input_data(msg)
    # Assert that the kafka_app responded with IrisPedictionData in predictions topic
    await tester.awaited_mocks.on_predictions.assert_awaited_with(IrisPredictionData(species="setosa"), timeout=2)

[INFO] fastkafka._testing.local_broker: Java is already installed.
[INFO] fastkafka._testing.local_broker: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Kafka is already installed.
[INFO] fastkafka._testing.local_broker: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: kafka started, sleeping for 5 seconds...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9092'}'
[INFO] fastkafka._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fastkafka._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting.

### Recap

We have created a Iris classification model and encapulated it into our fastkafka application.
The app will consume the IrisInputData from the `input_data` topic and produce the predictions to `predictions` topic.

To test the app we have:
1. Created the app
1. Started the LocalKafkaBroker
2. Started our Tester class which mirrors the developed app topics for testing purpuoses
3. Sent IrisInputData message to `input_data` topic
4. Asserted and checked that the developed iris classification service has reacted to IrisInputData message 

## Documentation

The kafka app comes with builtin documentation generation, let's demonstrate that.

To generate the documentation programatically you just need to call `.generate_docs()` method of your app. This will generate the *asyncapi* folder in relative path where all your documentation will be saved. 

In [ ]:
! npm install -g n
! n lts
! npm install -g npm@latest

npm ERR! code EACCESming idealTree Completed in 996ms
npm ERR! syscall mkdir
npm ERR! path /usr/lib/node_modules/n
npm ERR! errno -13
npm ERR! Error: EACCES: permission denied, mkdir '/usr/lib/node_modules/n'
npm ERR!  [Error: EACCES: permission denied, mkdir '/usr/lib/node_modules/n'] {
npm ERR!   errno: -13,
npm ERR!   code: 'EACCES',
npm ERR!   syscall: 'mkdir',
npm ERR!   path: '/usr/lib/node_modules/n'
npm ERR! }
npm ERR! 
npm ERR! The operation was rejected by your operating system.
npm ERR! It is likely you do not have the permissions to access this file as the current user
npm ERR! 
npm ERR! If you believe this might be a permissions issue, please double-check the
npm ERR! permissions of the file and its containing directories, or try running
npm ERR! the command again as root/Administrator.

npm ERR! A complete log of this run can be found in:
npm ERR!     /home/tvrtko/.npm/_logs/2023-03-03T17_10_43_175Z-debug-0.log
/bin/bash: line 1: n: command not found
npm ERR! code EACCESm

In [ ]:
! fastkafka docs install_deps

[INFO] fastkafka._components.asyncapi: AsyncAPI generator installed


In [ ]:
kafka_app_source = """
from pydantic import BaseModel, NonNegativeFloat, Field

class IrisInputData(BaseModel):
    sepal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal length in cm"
    )
    sepal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal width in cm"
    )
    petal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal length in cm"
    )
    petal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal width in cm"
    )


class IrisPredictionData(BaseModel):
    species: str = Field(..., example="setosa", description="Predicted species")
    
from fastkafka.application import FastKafka

kafka_app = FastKafka()

iris_species = ["setosa", "versicolor", "virginica"]

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = model.predict([
          [msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]
        ])[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPredictionData:
    prediction = IrisPredictionData(species=iris_species[species_class])
    return prediction
"""

In [ ]:
with open("application.py", "w") as source:
    source.write(kafka_app_source)

In [ ]:
! fastkafka docs generate application:kafka_app

[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/work/fastkafka/nbs/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /work/fastkafka/nbs/asyncapi/docs.




In [ ]:
! ls asyncapi/

docs  spec


In docs folder you will find the servable static html file of your documentation. This can also be served using our `fastkafka docs serve` CLI command (more on that in our guides).

In spec folder you will find a asyncapi.yml file containing the async API specification of your application. 
Lets view the generated docs now.

In [ ]:
from fastkafka.testing import display_docs

In [ ]:
await display_docs(docs_path="asyncapi/docs")

[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 370721...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 370721 terminated.
